In [1]:
import sys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import getpass

from IPython.display import Markdown
def printmd(string):
    display(Markdown(string))

pd.set_option('display.max_columns', 20) 
pd.set_option('display.max_rows', 1000)


#install Chromedrivermanager
#ChromeDriverManager().install()

### Navigate to web page

In [719]:
driver = webdriver.Chrome(executable_path='path of chromedriver')

#Open login page
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

# #Enter login info:
elementID = driver.find_element_by_id('username')
elementID.send_keys('bahuang23@gmail.com')

elementID = driver.find_element_by_id('password')
elementID.send_keys(getpass.getpass("Password: "))
#Note: replace the keys "username" and "password" with your LinkedIn login info

elementID.submit()


### Webpages

In [686]:
DA_url_past_wk = 'https://www.linkedin.com/jobs/search/?f_E=2%2C3&f_TPR=r604800&geoId=90000070&keywords=data%20analyst&location=New%20York%20City%20Metropolitan%20Area'
DA_url_past_mo = 'https://www.linkedin.com/jobs/search/?f_E=2%2C3&f_TPR=r2592000&geoId=90000070&keywords=data%20analyst&location=New%20York%20City%20Metropolitan%20Area'
BA_url_past_mo = 'https://www.linkedin.com/jobs/search/?f_E=2%2C3&f_TPR=r2592000&geoId=90000070&keywords=business%20analyst&location=New%20York%20City%20Metropolitan%20Area'
rec_jobs = 'https://www.linkedin.com/jobs/search/?currentJobId=2536632686&pivotType=jymbii'

### Go to url

In [782]:
driver.get(BA_url_past_mo)

### Main script

In [1085]:
#initialize empty list to include all jobs
job_desc_details_list = []

In [1086]:
next_page_to_scrape = 2
last_page_to_scrape = 4

for page in range(next_page_to_scrape, last_page_to_scrape + 2):    
    
    #Scrolling
    for _ in range(5):
        driver.execute_script("window.document.getElementsByClassName('jobs-search-results')[0].scrollBy(0, document.getElementsByClassName('jobs-search-results')[0].scrollHeight/4)") 
        time.sleep(0.2)

    for _ in range(2):
        for _ in range(3):
            driver.execute_script("window.document.getElementsByClassName('jobs-search-results')[0].scrollBy(0, -document.getElementsByClassName('jobs-search-results')[0].scrollHeight/4)")
            time.sleep(0.2)
        for _ in range(3):
            driver.execute_script("window.document.getElementsByClassName('jobs-search-results')[0].scrollBy(0, document.getElementsByClassName('jobs-search-results')[0].scrollHeight/4)")
            time.sleep(0.2)

    
    #Get web source
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    
    
    #All job tiles
    tiles = soup.find_all('li', class_ = "jobs-search-results__list-item occludable-update p0 relative ember-view")

    print("# of listings after scrolling:", len(tiles))
    
    #Extracting
    for i in tiles:
        #clicking
        id = i.find('div', class_ = "mr1 artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view").get('id')
        driver.find_element_by_id(id).click()
        time.sleep(0.5)
        

        job_desc_details = []
        
        
        
        #Job html source 
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        
        #Company name
        try:
            company_name = soup.find("a", class_="ember-view t-black t-normal").getText().strip()
        except:
            company_name = 'No company name'


        job_desc_details.append(company_name)
        
        
        #Title
        try:
            job_title = soup.find("h2", class_="t-24 t-bold").getText().strip()
            job_desc_details.append(job_title)
        except:
            continue
        
        
        #Location
        try: 
            location = soup.find("span", class_="jobs-unified-top-card__bullet").getText().strip()
        except:
            continue

        job_desc_details.append(location)
        
        
        #Posted date

        posted_date = soup.find("span", class_="jobs-unified-top-card__posted-date").getText().strip()
        job_desc_details.append(posted_date)

        
        
        #Applicants, Level, Company size, Industry, Connections
        
        try:
            applicants_no = soup.find("span", class_="jobs-unified-top-card__applicant-count").getText().strip()
        except:
            applicants_no = soup.find_all("span", class_="jobs-unified-top-card__bullet")[1].getText().strip()
        
        level_compsize_industry_connections = soup.find_all("div", class_="jobs-unified-top-card__job-insight")
        
        
        try:
            level = level_compsize_industry_connections[0].getText().strip()
            comp_size_and_industry = level_compsize_industry_connections[1].getText().strip()
        except:
            level = "Couldn't get level"
            comp_size_and_industry = 'Company page not claimed'
            
        if len(level_compsize_industry_connections) > 3:    
            connections = level_compsize_industry_connections[2].getText().strip()
        else:
            connections = 'No connections'
        
        
        job_desc_details.extend([applicants_no, level, comp_size_and_industry, connections])
        
                
        
        #JD
        job_description = soup.find("div", class_="jobs-box__html-content jobs-description-content__text t-14 t-normal").getText().strip()

        job_desc_details.append(job_description)
        
        
        #URL
        job_desc_details.append(driver.current_url)
        
        
        #Adding all job info to list
        job_desc_details_list.append(job_desc_details)
    
    
    #Click next page
    try:
        driver.find_element_by_xpath(f"//ul[@class='artdeco-pagination__pages artdeco-pagination__pages--number']/li[@data-test-pagination-page-btn = '{page}']").click()
    except:
        pass
    time.sleep(1.5)

    
    
print("list_total:", len(job_desc_details_list))

# of listings after scrolling: 25
# of listings after scrolling: 20
# of listings after scrolling: 25
# of listings after scrolling: 25
list_total: 95


### Putting jobs info into dataframe

In [1087]:
job_desc_details_list_df = pd.DataFrame(job_desc_details_list, columns = ['Company Name', 'Job Title', 'Location', 'Posted_date', '# of applicants', 'Level', 'Company size and Industry', 
                                                                          'Connections', 'Job Description', 'url'])

job_desc_details_list_df = job_desc_details_list_df.drop_duplicates(subset = ['Company Name', 'Job Title', 'Location', 'Posted_date', '# of applicants', 'Level', 'Company size and Industry', 'url']).reset_index(drop=True)
print(job_desc_details_list_df.shape)

(95, 10)


In [812]:
job_desc_details_list_df.head(10)

,Company Name,Job Title,Location,Views,Posted_date,# of applicants,Level,Company size,Industry,Connections,Job Description,url
0,Greenlots,Business Development Analyst - East Coast,"New York, NY",115 views,Posted 3 weeks ago,12 applicants,Associate,51-200 employees,Automotive,You have 0 connections at this company.,Business Development Analyst – East Coast\n\nJ...,https://www.linkedin.com/jobs/search/?currentJ...
1,"Ross Stores, Inc.","Business Analyst, Business Information","New York, NY","1,332 views",Posted 3 weeks ago,600 applicants,Associate,10001+ employees,Retail,[2 alumni],Posted by\n \n\n\n\n\n\n\n\n\n D...,https://www.linkedin.com/jobs/search/?currentJ...
2,TEKsystems,Business Systems Analyst,"New York, NY",33 views,Posted 1 week ago,5 applicants,Associate,"1,001-5,000 employees",Information Technology & Services,[8 alumni],Summary\n\nThe main function of a GBS business...,https://www.linkedin.com/jobs/search/?currentJ...
3,WeWork,"Associate, Data Analyst","New York, NY",597 views,Posted 14 hours ago,171 applicants,Associate,"5,001-10,000 employees",Real Estate,[11 alumni],WeWork Mission\n\nWeWork is the platform for c...,https://www.linkedin.com/jobs/search/?currentJ...
4,Peloton Interactive,"Data/Reporting Analyst, IT Service Design","New York, NY",275 views,Posted 1 day ago,45 applicants,Associate,"1,001-5,000 employees","Health, Wellness & Fitness","[3 company alumni, 19 alumni]",Peloton’s IT organization is seeking an experi...,https://www.linkedin.com/jobs/search/?currentJ...
5,Etsy,Business Development Analyst,"Brooklyn, NY",266 views,Posted 1 week ago,46 applicants,Associate,"1,001-5,000 employees",Internet,"[1 connection, 2 company alumni]",Company Description\n\nEtsy is the global mark...,https://www.linkedin.com/jobs/search/?currentJ...
6,Feather,Business Intelligence Analyst,"New York, NY",326 views,Posted 2 weeks ago,83 applicants,Associate,51-200 employees,Furniture,You have 0 connections at this company.,"At Feather, our mission is to transform people...",https://www.linkedin.com/jobs/search/?currentJ...
7,"Glossier, Inc.",Program Analyst,"New York, NY",457 views,Posted 4 days ago,26 applicants,Associate,201-500 employees,Consumer Goods,[1 alum],Overview\n\nThe Programs Business Analyst II w...,https://www.linkedin.com/jobs/search/?currentJ...
8,Nomura,Data Management Analyst,New York City Metropolitan Area,517 views,Posted 1 day ago,239 applicants,Associate,10001+ employees,Financial Services,"[1 connection, 19 alumni]",Company overview\nNomura is an Asia-based fina...,https://www.linkedin.com/jobs/search/?currentJ...
9,WebMD,Associate Business/Data Analyst,"New York, NY",449 views,Posted 2 weeks ago,108 applicants,Associate,"1,001-5,000 employees",Internet,[4 alumni],Jobson Healthcare Information (JHI) is a premi...,https://www.linkedin.com/jobs/search/?currentJ...


### Filter

In [1088]:
#Specify words to remove
words_to_remove = ['3 years', '3+ years', '4+ years', '5+ years', '5 years', '6 years', '10+ years', '10 years', '6+ years', '3-5+ years', 'Five or more years', 'Six or more years', 'Three or more years', 'four or more years',
                'Five plus years', 'five or more years', 'six or more years', 'three or more years', 'Four or more years', '3-5 years', '8 years', '4 years as an individual contributor', '8+ year', '3+ experience',
                  '3+ year', '4 plus years', 'junior', 'Junior', '4 - 7 years', '7 years of experience']


consider_indexes = []

for i in range(len(job_desc_details_list_df)):
    jd = job_desc_details_list_df.loc[i, 'Job Description']
    if any(x in jd for x in words_to_remove):
        consider_indexes.append(i)
        
consider_jobs = job_desc_details_list_df[~job_desc_details_list_df.index.isin(consider_indexes)].reset_index(drop=True)
print(consider_jobs.shape)

(55, 10)


#### Filter industry

In [975]:
industries_to_filter_out = ['Management Consulting', 'Airlines/Aviation']

consider_jobs = consider_jobs[~consider_jobs['Industry'].isin(industries_to_filter_out)]

consider_jobs.reset_index(drop=True, inplace=True)

#### Lookup posting

In [1089]:
consider_jobs.loc[2].values

array(['WarnerMedia', 'Analyst, Research - HBO Max', 'New York, NY',
       '2 hours ago', '23 applicants', 'Full-time · Associate',
       '10,001+ employees · Entertainment',
       '13 company alumni · 13 school alumni',
       'PeopleSoft Requisition Number\nR000037432\n\n Company Overview \n\nWarnerMedia is a leading media and entertainment company that creates and distributes premium and popular content from a diverse array of talented storytellers and journalists to global audiences through its consumer brands including: HBO, HBO Max, Warner Bros., TNT, TBS, truTV, CNN, DC Entertainment, New Line, Cartoon Network, Adult Swim, Turner Classic Movies and others.\n\n Business Unit Overview \n\nHBO Max is the future of entertainment and storytelling.\u202fWarnerMedia’s streaming entertainment offering, HBO Max launched in May 2020 and is the culmination of some of the most innovative new technology and greatest creative talent in the industry. Anchored by the entire HBO service, argu

### Print filtered jobs dataframe

In [1094]:
consider_jobs.head(10)

,Company Name,Job Title,Location,Posted_date,# of applicants,Level,Company size and Industry,Connections,Job Description,url
0,Creative Circle,Analyst,"New York, NY",28 minutes ago,"New York, NY",Full-time · Entry level,201-500 employees · Staffing & Recruiting,1 school alumni,Our agency client is seeking an Analyst to fun...,https://www.linkedin.com/jobs/search/?currentJ...
1,Affinity Solutions,Business Data Analyst,"New York, NY",6 hours ago,36 applicants,Full-time · Associate,51-200 employees · Marketing & Advertising,No connections,Business/Data Analyst Affinity Solutions is th...,https://www.linkedin.com/jobs/search/?currentJ...
2,WarnerMedia,"Analyst, Research - HBO Max","New York, NY",2 hours ago,23 applicants,Full-time · Associate,"10,001+ employees · Entertainment",13 company alumni · 13 school alumni,PeopleSoft Requisition Number\nR000037432\n\n ...,https://www.linkedin.com/jobs/search/?currentJ...
3,Harnham,Data Analyst,New York City Metropolitan Area,4 hours ago,157 applicants,Full-time · Entry level,51-200 employees · Staffing & Recruiting,No connections,Posted by\n \n\n\n\n\n\n\n\n\n O...,https://www.linkedin.com/jobs/search/?currentJ...
4,JetBlue,Engineer Crew Analytics,"Long Island City, New York, United States",48 minutes ago,1 applicant,Couldn't get level,Company page not claimed,No connections,Position Summary\n\nThe position is responsibl...,https://www.linkedin.com/jobs/search/?currentJ...
5,Scouted,Data Insights Analyst,New York City Metropolitan Area,1 hour ago,32 applicants,Full-time · Entry level,11-50 employees · Human Resources,No connections,The hiring process for this role is managed by...,https://www.linkedin.com/jobs/search/?currentJ...
6,The Topps Company,Data Analyst,"New York, NY",12 hours ago,55 applicants,Full-time · Entry level,201-500 employees · Consumer Goods,No connections,"Overview\n\nFounded in 1938, The Topps Company...",https://www.linkedin.com/jobs/search/?currentJ...
7,LeafLink,Data Analyst,"New York, NY",1 week ago,136 applicants,Full-time · Entry level,51-200 employees · Computer Software,No connections,About LeafLink\n\nLeafLink is the regulated ca...,https://www.linkedin.com/jobs/search/?currentJ...
8,Cornershop by Uber,Data Analyst,New York City Metropolitan Area,4 days ago,114 applicants,Full-time · Associate,"1,001-5,000 employees · Internet",No connections,Posted by\n \n\n\n\n\n\n\n\n\n C...,https://www.linkedin.com/jobs/search/?currentJ...
9,American Express,"Product Analyst, Customer 360","New York, NY",17 hours ago,21 applicants,"$63,000/yr - $95,000/yr (est.) · Full-time · A...","10,001+ employees · Financial Services",1 connection · 1 company alumni · 48 school al...,You Lead the Way. We’ve Got Your Back.\n\nAt A...,https://www.linkedin.com/jobs/search/?currentJ...


#### Print out postings

In [1093]:
for i in range(len(consider_jobs[5:11])):
    
    company_name = consider_jobs.iloc[i, 0]
    job_title = consider_jobs.iloc[i, 1]
    location = consider_jobs.iloc[i, 2]
    posted_date = consider_jobs.iloc[i, 3]
    no_applicants = consider_jobs.iloc[i, 4]
    level = consider_jobs.iloc[i, 5]
    company_size_and_industry = consider_jobs.iloc[i, 6]
    connections = consider_jobs.iloc[i, 7]
    job_description = consider_jobs.iloc[i, 8]
    url = consider_jobs.iloc[i, 9]
    
    printmd('**'+ company_name +'**')
    printmd('**'+ job_title +'**')
    printmd('**'+ location +'**')
    printmd('**'+ posted_date +'**')
    printmd('**'+ no_applicants +'**')
    printmd('**'+ level +'**')
    printmd('**'+ company_size_and_industry +'**')
    printmd('**'+ connections +'**')
    printmd('**'+ job_description +'**')
    print()
    print('URL:', url)
    printmd('*'*500)
    printmd('\n')

**Creative Circle**

**Analyst**

**New York, NY**

**28 minutes ago**

**New York, NY**

...

**11-50 employees · Human Resources**

**No connections**

**The hiring process for this role is managed by Scouted, a job-matching platform that believes people are more than their resumes. In order to be considered, click the ‘Apply’ button. If you have an urgent question, email scouted@scouted.io. Please do NOT use InMail. You will not get a response.----------Role OverviewThis boutique hedge fund is looking for a Data Insights Analyst to join their team. The Data Insights Analyst will be responsible for providing data driven insights in response to a variety of investment questions. This position reports to the Head of Data and Analytics and is part of a growing data team with responsibilities spanning from data sourcing and processing, to analytics.
In addition to providing insights on a tactical level, the Data Insights Analyst will help build bespoke machine learning models to enable processing (e.g., sentiment analysis, product classification) and prediction at scale. There will also be close collaboration with an established Field Research team that primarily focuses on qualitative research.
The ideal candidate is an entrepreneurial self-starter who is passionate about continuous learning in a rapidly evolving data analytics space. Extreme intellectual curiosity and attention to detail, while being comfortable and flexible in a growing organization, are essential.
What will you do?Acquiring high quality data: identifying potential data vendors and/ or spearheading custom in-house alt-data acquisition (e.g., data from survey/ crowdsourcing); backtesting data against groundtruth; understanding methodology and limitations of data collection; developing custom solutions collaboratively with vendors; liaising with internal Legal and Compliance functions; project managing trial and onboarding processes; manual small-scale data gathering (e.g., country-specific demographics data); cleaning data (anomaly checks/ entity mapping) either independently or with external vendorsManaging relationship with investment analysts: proactively identifying where data may add value to investment theses; brainstorming creative approaches for tactical investigations; showcasing new data sources and use cases; advocating a data-driven culture throughout the organizationDelivering bespoke insights: conceptualizing framework for investigation; cleaning and augmenting data with custom features to facilitate analyses; refining analyses based on iterative feedback; developing and maintaining reports and interactive Tableau dashboards; ensuring the data and analysis on Tableau dashboards are current; crystalizing the “so-whats” of analyses; challenging investment analysts in their assumptionsLeading strategic capability development: continuous researching of new tools and processes to increase team’s efficiency and capabilities; collaborating with Field Research team to identify synergies; conducting post-project and regular retrospectives to propose improvements; developing bespoke tools or domain-specific models; coding surveys on platforms such as SurveyGizmo; testing surveys before launch; liaising with vendors to ensure high quality execution (e.g., checking representativeness, response quality)Managing project flow for Data team: maintaining pipeline of data due-diligence projects to assist with prioritization, tracking and reportingData library: maintaining a catalog of current and potential data sources, together with associated meta-data, to facilitate discovery across the firmTracking data and system costs: working with Finance team to keep track of variable (project based) and fixed costs for the Data effortTranslation support: working with external agencies to translate surveys and other data assetsWhat are you like?Ideal Candidates Will Have:Bachelor’s degree with a minimum 3.6 GPA from a Tier 1 universityEntrepreneurial track record, either from founding or being a member of a founding teamHighly numeric and detail oriented, as well as skilled at creative problem solvingExcellent interpersonal and communication skillsTrack record of continuous learningDemonstrated passion for data, although past experience not requiredHighest degree of integrity, professionalism and confidentialityFit with the company's Core PrinciplesMajor plus, but not required:Previous experience with surveys, from questionnaire development to analysis and reportingPrevious experience with big data preparation (e.g., Alteryx) and visualization (e.g., Tableau) toolsProgramming experience, esp. PythonProficiency in Mandarin or Spanish**


URL: https://www.linkedin.com/jobs/search/?currentJobId=2579835832&f_E=2%2C3&f_TPR=r2592000&geoId=90000070&keywords=data%20analyst&location=New%20York%20City%20Metropolitan%20Area


********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************